<a href="https://colab.research.google.com/github/Elma-dev/Deep_Learning_Specialization/blob/main/Deep_Learning_Specialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import time

# Vectorization

In [6]:
#Sum Of Column Without for loop
A=np.array([[65,0,4.4,68]
         ,[1.2,104,52,8]
         ,[1.8,135,99,0.9]])

ones=np.ones((1,3))
np.dot(ones,A)

array([[ 68. , 239. , 155.4,  76.9]])

In [17]:
#Vectorization Vs forLoop
A=np.random.rand(10000)
B=np.random.rand(10000)

#Vectorization
tic=time.time()
c=np.dot(A,B)
tac=time.time()
print(f"result={c} the operation take: {(tac-tic)*1000}ms")

#For Loop
c=0
tic=time.time()
for i in range(10000):
  c+=A[i]*B[i]
tac=time.time()
print(f"result={c} the operation take: {(tac-tic)*1000}ms")

result=2471.8928285555025 the operation take: 0.1277923583984375ms
result=2471.8928285554866 the operation take: 4.808187484741211ms


# Trics

In [26]:
#Row Vector / Column Vector / Rank 1 array

#Rank 1 array
print(f"rank_1_array:\n{np.random.randn(5)}") #-> don't use it
print(f"column_vector:\n{np.random.randn(1,5)}")
print(f"row_vector:\n{np.random.randn(5,1)}")

rank_1_array:
[ 0.1241815  -0.98621466 -0.72945895 -0.99423603  0.45756124]
column_vector:
[[-0.4032663  -1.52366816  2.576655   -0.5910431   0.52406475]]
row_vector:
[[ 1.67827423]
 [-1.18179114]
 [-0.79054254]
 [-0.44496086]
 [ 1.13109552]]


In [23]:
a=np.array([[2,1],[1,3]])
np.dot(a,a)

array([[ 5,  5],
       [ 5, 10]])

In [30]:
a=np.random.randn(12288,150)
b=np.random.randn(150,45)

np.dot(a,b).shape
a.sum()

(12288, 45)

In [28]:
a=np.random.randn(2,3)
b=np.random.randn(2,1)
a+b

array([[-0.77279559, -0.90674973,  0.36632089],
       [ 0.11768284,  2.26659315, -3.07627972]])

# Cat Classification Scratch